In [1]:
import sqlite3
import nest_asyncio
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from catboost import CatBoostClassifier
import pandas as pd

bot = Bot('7629247815:AAFDwohea1Y3blp8eHo1frRdIbKN6LVMN_s')
nest_asyncio.apply()

In [ ]:
class Mystate(StatesGroup):
    register = State()
    survey_start = State()
    survey = State()




conn = sqlite3.connect('database.sql')
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS clients (id INTEGER PRIMARY KEY, name TEXT, email TEXT)')
cur.execute('''CREATE TABLE IF NOT EXISTS survey_results 
             (id INTEGER PRIMARY KEY, client_id INTEGER, age REAL, 
             ap_hi INTEGER, ap_lo INTEGER, cholesterol TEXT, 
             gluc TEXT, height INTEGER, weight INTEGER,
             prediction INTEGER, 
             FOREIGN KEY(client_id) REFERENCES clients(id))''')
conn.commit()






model = CatBoostClassifier()
model.load_model('../models/final_model.cbm')





start_markup = ReplyKeyboardMarkup(resize_keyboard=True)
start_markup.add(KeyboardButton('/start'))

survey_markup = ReplyKeyboardMarkup(resize_keyboard=True)
survey_markup.add(KeyboardButton('Yes'), KeyboardButton('No'))

cholesterol_markup = ReplyKeyboardMarkup(resize_keyboard=True)
cholesterol_markup.add(KeyboardButton('normal'), 
                      KeyboardButton('above normal'), 
                      KeyboardButton('well above normal'))

gluc_markup = ReplyKeyboardMarkup(resize_keyboard=True)
gluc_markup.add(KeyboardButton('normal'), 
               KeyboardButton('above normal'), 
               KeyboardButton('well above normal'))


storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)



@dp.message_handler(commands=['start'])
async def start(message: types.Message):
    await message.answer('''
Hi! I am a medical bot for cardiovascular risk assessment.
I can predict your risk and give recommendations.

Enter the data in the format:
Name: your name
Email: your email
''', reply_markup=ReplyKeyboardRemove())
    await Mystate.register.set()

@dp.message_handler(state=Mystate.register)
async def register_user(message: types.Message, state: FSMContext):
    try:
        data = message.text.split('\n')
        name = data[0].split(':')[1].strip()
        email = data[1].split(':')[1].strip()

        cur.execute("INSERT INTO clients (name, email) VALUES (?, ?)", (name, email))
        conn.commit()
        client_id = cur.lastrowid
        await state.update_data(client_id=client_id)

        await message.answer(f'Registration completed, {name}!')
        await message.answer('Ready to start the survey?', reply_markup=survey_markup)
        await Mystate.survey_start.set()

    except Exception as e:
        await message.answer(f'Error: {e}\nEnter data in the format:\nName: your name\nEmail: your email')





@dp.message_handler(state=Mystate.survey_start)
async def confirm_start_survey(message: types.Message, state: FSMContext):
    if message.text.lower() == 'yes':
        await state.update_data(current_question=0, answers={})
        await Mystate.survey.set()
        await ask_next_question(message, state)
    else:
        await message.answer('Okay, if you change your mind, press /start', reply_markup=start_markup)
        await state.finish()




questions = [
    "1. Enter your systolic blood pressure (top number, mmHg):",
    "2. Enter your age:",
    "3. Select your cholesterol level:",
    "4. Enter your diastolic blood pressure (bottom number, mmHg):",
    "5. Enter your weight (in kg):",
    "6. Enter your height (in cm):",
    "7. Select your blood glucose level:"
]



async def ask_next_question(message: types.Message, state: FSMContext):
    data = await state.get_data()
    current_question = data['current_question']

    if current_question < len(questions):
        if current_question == 2:
            await message.answer(questions[current_question], reply_markup=cholesterol_markup)
        elif current_question == 6:
            await message.answer(questions[current_question], reply_markup=gluc_markup)
        else:
            await message.answer(questions[current_question], reply_markup=ReplyKeyboardRemove())
    else:
        await process_survey_results(message, state)




@dp.message_handler(state=Mystate.survey)
async def handle_answer(message: types.Message, state: FSMContext):
    data = await state.get_data()
    current_question = data['current_question']
    answers = data['answers']

    try:
        if current_question == 0:
            val = int(message.text)
            if not 50 <= val <= 250:
                raise ValueError("Enter pressure from 50 to 250")
            answers['ap_hi'] = val

        elif current_question == 1:
            val = float(message.text)
            if not 10 <= val <= 120:
                raise ValueError("Age must be between 10 and 120 years")
            answers['age'] = val

        elif current_question == 2:
            val = message.text.lower()
            if val not in ['normal', 'above normal', 'well above normal']:
                raise ValueError("Select from keyboard")
            answers['cholesterol'] = val

        elif current_question == 3:
            val = int(message.text)
            if not 30 <= val <= 150:
                raise ValueError("Enter pressure from 30 to 150")
            answers['ap_lo'] = val

        elif current_question == 4:
            val = float(message.text)
            if not 20 <= val <= 300:
                raise ValueError("The weight should be from 20 to 300 kg")
            answers['weight'] = val

        elif current_question == 5:
            val = int(message.text)
            if not 50 <= val <= 250:
                raise ValueError("Height should be from 50 to 250 cm")
            answers['height'] = val

        elif current_question == 6:
            val = message.text.lower()
            if val not in ['normal', 'above normal', 'well above normal']:
                raise ValueError("Select from keyboard")
            answers['gluc'] = val

        await state.update_data(answers=answers, current_question=current_question + 1)
        await ask_next_question(message, state)

    except Exception as e:
        await message.answer(f"Eror: {e}")





async def process_survey_results(message: types.Message, state: FSMContext):
    data = await state.get_data()
    client_id = data['client_id']
    answers = data['answers']

    cholesterol_map = {'normal': 1, 'above normal': 2, 'well above normal': 3}
    gluc_map = {'normal': 1, 'above normal': 2, 'well above normal': 3}

    features = [
        answers['age'],
        answers['ap_hi'],
        cholesterol_map[answers['cholesterol']],
        answers['ap_lo'],
        gluc_map[answers['gluc']],
        answers['height'],
        answers['weight']
    ]

    input_df = pd.DataFrame([features], columns=[
        'age_new', 'ap_hi', 'cholesterol', 'ap_lo', 'gluc', 'height', 'weight'
    ])

    prediction = model.predict(input_df)[0]

    cur.execute('''INSERT INTO survey_results 
        (client_id, age, ap_hi, ap_lo, cholesterol, gluc, height, weight, prediction)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''',
        (client_id, answers['age'], answers['ap_hi'], answers['ap_lo'],
         answers['cholesterol'], answers['gluc'], answers['height'],
         answers['weight'], prediction))
    conn.commit()

    if prediction == 1:
        msg = '''You have an increased risk.

Recommendations:
1) Lose weight
2) Try to move more or do sports
3) Reduce your intake of salt and sugar
4) Be sure to see a doctor
5) Stop smoking and drinking, if you do these
        '''
    else:
        msg = ''' Your risk is within the normal range.

Recommendations:
1) Maintain your current lifestyle
2) Monitor your blood pressure annually
3) Healthy diet and activity
        '''

    await message.answer(msg, reply_markup=start_markup)
    await state.finish()





if __name__ == '__main__':
    executor.start_polling(dp, skip_updates=True)

In [ ]:
Feature Id	Importances
0	ap_hi	52.547054
1	age_new	19.715230
2	cholesterol	11.517713
3	ap_lo	6.775532
4	weight	5.420534
5	height	2.156967
6	gluc	1.866970


In [3]:
cur.execute("SELECT * FROM clients")
rows = cur.fetchall()

for row in rows:
    print(row)

(1, 'Platon ', 'platon12345')
(2, 'Platon ', 'platon12345')
(3, 'Platon', 'platon95@icloud.com')
(4, 'Platon', 'platon95@icloud.com')
(5, 'Platon', 'platon95@icloud.com')
(6, 'Platon', 'platon95@icloud.com')
(7, 'Platon', 'platon95@icloud.com')
(8, 'Platon', 'platon95@icloud.com')
(9, 'Platon', 'platon95@icloud.com')
(10, 'Platon', 'platon95@icloud.com')
(11, 'Platon', 'platon95@icloud.com')
(12, 'Platon', 'platon95@icloud.com')
(13, 'Эмин', 'added')
(14, 'User', 'User@gmail.com')
(15, 'Илья', 'pretty')
(16, 'Илья', 'pretty')
(17, 'Илья', 'pretty')
(18, 'Илья', 'pretty')


Cause exception while getting updates.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/aiogram/bot/api.py", line 139, in make_request
    async with session.post(url, data=req, **kwargs) as response:
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/aiohttp/client.py", line 1167, in __aenter__
    self._resp = await self._coro
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/aiohttp/client.py", line 586, in _request
    await resp.start(conn)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 905, in start
    message, payload = await protocol.read()  # type: ignore[union-attr]
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/aiohttp/streams.py", line 616, in read
    await self._waiter
  File "/Library/Frameworks/Python.framework/Versions/

In [4]:
cur.execute("SELECT * FROM survey_results")
rows = cur.fetchall()

for row in rows:
    print(row)

(1, 11, 20.5, None, 182, 79, 110, 60, 'normal', 'normal', None, None, None, 0)
(2, 12, 60.7, None, 170, 100, 200, 100, 'well above normal', 'above normal', None, None, None, 1)
(3, 13, 25.1, None, 180, 93, 110, 60, 'above normal', 'normal', None, None, None, 0)
(4, 14, 10, None, 158, 69, 120, 80, 'normal', 'above normal', None, None, None, 0)
(5, 15, 27.2, None, 182, 82, 130, 70, 'above normal', 'above normal', None, None, None, 0)
(6, 16, 50.2, None, 180, 100, 180, 110, 'above normal', 'well above normal', None, None, None, 1)
(7, 17, 19, None, 180, 75, 60, 40, 'normal', 'normal', None, None, None, 0)
(8, 18, 52, None, 167, 63, 130, 80, 'normal', 'normal', None, None, None, b'\x00\x00\x00\x00\x00\x00\x00\x00')
